In [1]:
""" Starter code for logistic regression model to solve OCR task 
with MNIST in TensorFlow
MNIST dataset: yann.lecun.com/exdb/mnist/
Author: Chip Huyen
Prepared for the class CS 20SI: "TensorFlow for Deep Learning Research"
cs20si.stanford.edu
"""
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import time

# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 10

# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('/data/mnist', one_hot=True) 

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /data/mnist\train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /data/mnist\train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /data/mnist\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /data/mnist\t10k-labels-idx1-ubyte.gz


In [13]:
# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 
# Features are of the type float, and labels are of the type int
X = tf.placeholder(tf.float32, shape = [batch_size,784], name = "Features")
Y = tf.placeholder(tf.int32, shape = [batch_size,10], name="labels")

# set batch_size here is important!



# Step 3: create weights and bias
# w is initialized to random variables with mean of 0, stddev of 0.01
# biase is initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y
W = tf.Variable(tf.random_normal(shape = [784, 10], stddev = 0.01),  name = "weights")
b = tf.Variable(tf.zeros([10]), name = "bias")

# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE


# because here is a matrix rather than a number (vector)
logits = tf.matmul(X, W) + b

# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch
entropy = tf.nn.softmax_cross_entropy_with_logits(logits= logits, labels = Y)
loss = tf.reduce_mean(entropy)

# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [18]:
n_batches

78

In [19]:
with tf.Session() as sess:
	start_time = time.time()
	sess.run(tf.global_variables_initializer())
	n_batches = int(mnist.train.num_examples/batch_size)
	for i in range(n_epochs): # train the model n_epochs times
		total_loss = 0

		for i in range(n_batches):
			X_batch, Y_batch = mnist.train.next_batch(batch_size)
			# TO-DO: run optimizer + fetch loss_batch
			_, loss_batch = sess.run([optimizer, loss], feed_dict= {X: X_batch, Y:Y_batch}) 
			total_loss += loss_batch
		print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))

	print('Total time: {0} seconds'.format(time.time() - start_time))

	print('Optimization Finished!') # should be around 0.35 after 25 epochs

	# test the model
	preds = tf.nn.softmax(logits)
	correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y, 1))
	accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
	
	n_batches = int(mnist.test.num_examples/batch_size)
	total_correct_preds = 0
	
	for i in range(n_batches):
		X_batch, Y_batch = mnist.test.next_batch(batch_size)
		accuracy_batch = sess.run([accuracy], feed_dict={X: X_batch, Y:Y_batch}) 
		total_correct_preds += sum(accuracy_batch)
	
	print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 428: 1.2910309988301951
Average loss epoch 428: 0.7345871967035573
Average loss epoch 428: 0.6016601737423655
Average loss epoch 428: 0.536774122562164
Average loss epoch 428: 0.4982892374197642
Average loss epoch 428: 0.4719742194061235
Average loss epoch 428: 0.450600496747277
Average loss epoch 428: 0.4369937475327845
Average loss epoch 428: 0.4230210990199954
Average loss epoch 428: 0.4139370822406315
Total time: 4.657084226608276 seconds
Optimization Finished!
Accuracy 0.8978
